# Modelo Supervisionado

&nbsp;&nbsp;&nbsp;&nbsp; Um modelo supervisionado é responsável por receber dados rotulados e desenvolver um algoritmo que é capaz de prever e classificar dados. Os modelos supervisionados se dividem em duas vertentes: Classificação e regressão. A classificação, como o nome já diz, é responsável por classificar e criar grupos de dados com características semelhantes. Já a regressão é utilizada para fazer previsões e avaliar variáveis isoladas. [(Alteryx, 2023)](https://www.alteryx.com/pt-br/glossary/supervised-vs-unsupervised-learning)

#### Organização dos dados:

&nbsp;&nbsp;&nbsp;&nbsp; Para o modelo supervisionado, organizamos os dados da seguinte forma: 80% do banco de dados voltado para o treinamento e 20% voltado ao teste do modelo. Utilizamos essa disposição dos dados, pois assim temos uma boa quantidade de dados para o treinamento e uma boa margem para teste, além de previnirmos o *overfitting*, já que uma parte dos dados está separada especificamente para o teste do modelo.

&nbsp;&nbsp;&nbsp;&nbsp; O presente _notebook_ teve a finalidade de testar possíveis modelos preditivos que pudessem prever padrões de consumo dos clientes da base de dados com base no cosumo dos meses anteriores. Para isso, foram feitas adequações necessárias nos valores e testes consecutivos de diferentes modelos para então, escolher qual o mais satisfatório. Em decorrência dessa análise, as métricas mais satisfatórias vieram do modelo _**Regressão (SVR)**_ do _Scikit-Learn_.

# Imports

&nbsp;&nbsp;&nbsp;&nbsp; Importação das bibliotecas necessárias para análise e modelagem dos dados.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif, f_regression
from category_encoders import OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor, IsolationForest
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from scipy.stats import uniform, randint
from xgboost import XGBRegressor
import optuna
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D
from sklearn.datasets import make_blobs
import joblib
import matplotlib.pyplot as plt
import statsmodels.api as sm
import lightgbm as lgb
from sklearn.metrics import make_scorer, mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score, KFold


# Import datas

&nbsp;&nbsp;&nbsp;&nbsp; Importação dos dados que serão utilizados.

In [ ]:
df = pd.read_csv('../data/df_atualizado.csv')

# Seleção de features

&nbsp;&nbsp;&nbsp;&nbsp; Seleção dos dados do cliente com `clientCode` 683 e `clientIndex` 0.

In [ ]:
# pega o client com o clientcode 2 e o clientindex 0
df_client = df[(df['clientCode'] == 683) & (df['clientIndex'] == 0)]
df_client.head(5)

# df_client = df

&nbsp;&nbsp;&nbsp;&nbsp; Com o objetivo de visualizar a correlação entre as features para a construção do modelo preditivo, foi realizada uma matriz de correlação.

In [ ]:
df_client['date'] = pd.to_datetime(df_client['date'])

# Verificar a correlação entre as variáveis
plt.figure(figsize=(12, 8))
sns.heatmap(df_client.corr(), annot=True, cmap='coolwarm')
plt.title('Matriz de Correlação')
plt.show()

# Verificar a distribuição das variáveis
df.hist(bins=30, figsize=(20, 15))
plt.show()

&nbsp;&nbsp;&nbsp;&nbsp; A análise do gráfico acima revela que, em geral, as variáveis têm pouca correlação, como observado na coluna 'hora', que mostra uma correlação baixa, esse fato sugere que sua influência no comportamento do consumidor pode ser limitada ou não linear. Por outro lado, features como 'mês' e 'data' apresentam correlações significativas que podem ser exploradas para uma melhor compreensão do comportamento do cliente e suas sazonalidades.

&nbsp;&nbsp;&nbsp;&nbsp; Considerando que não é lógico prever o consumo utilizando valores negativos, que representam possíveis anomalias, as linhas com gastos negativos foram excluídas.

In [ ]:
# apaga todas as linhas com gasto 0
df_client = df_client[df_client['gasto'] > 0]


&nbsp;&nbsp;&nbsp;&nbsp; A célula abaixo comprova que todos os valores negativos foram removidos do *DataFrame*.

In [ ]:
teste = df_client[df_client['gasto'] < 0]
teste.head(10)

&nbsp;&nbsp;&nbsp;&nbsp; Cálculo da média e desvio padrão do consumo e remoção de valores fora do desvio padrão.

In [ ]:
# Identifica o consumo medio do cliente e o desvio padrao
media = df_client['gasto'].mean()
desvio = df_client['gasto'].std()

# Remove os consumos que estao fora do desvio padrao

maximo = media + (1*desvio)
df_client = df_client[df_client['gasto'] < maximo]

minimo = media - (1*desvio)
df_client = df_client[df_client['gasto'] > minimo]


&nbsp;&nbsp;&nbsp;&nbsp; Separação dos dados em variáveis independentes (X) e variável dependente (y).

In [ ]:
# Separar em recursos (X) e alvo (y)
X = df_client[['dia_da_semana', 'mes', 'feriado', 'Temp. Ins. (C)',
       'Temp. Max. (C)', 'Temp. Min. (C)', 'Umi. Ins. (%)', 'Umi. Max. (%)',
       'Umi. Min. (%)', 'Pto Orvalho Ins. (C)', 'Pto Orvalho Max. (C)',
       'Pto Orvalho Min. (C)', 'Pressao Ins. (hPa)', 'Pressao Max. (hPa)',
       'Pressao Min. (hPa)', 'Vel. Vento (m/s)', 'Dir. Vento (°)',
       'Raj. Vento (m/s)', 'Chuva (mm)']]

y = df_client['gasto']  

&nbsp;&nbsp;&nbsp;&nbsp; O gráfico plotado abaixo representa o gasto por dia da semana do cliente 683, conforme especificado anteriormente.

In [ ]:
# Grafico gasto x dia da semana
plt.figure(figsize=(12, 6))
df_client['dia_da_semana'] = df_client['dia_da_semana'].replace({0: 'Segunda', 1: 'Terça', 2: 'Quarta', 3: 'Quinta', 4: 'Sexta', 5: 'Sabado', 6: 'Domingo'})
sns.lineplot(x='dia_da_semana', y='gasto', data=df_client)
plt.title('Consumo de gás de um cliente por dia da semana - ClientCode 683')
plt.xlabel('Dia da Semana')
plt.ylabel('Consumo de gás (m3 - metros cúbicos)')
plt.legend(['Média de consumo de gás', 'Variância de consumo de gás'])
plt.show()

&nbsp;&nbsp;&nbsp;&nbsp; A análise do gráfico acima revela uma alta variabilidade no consumo de gás durante os dias da semana. Nesse sentido, em um mesmo dia da semana, pode haver tanto um consumo elevado quanto um consumo baixo. Nesse sentido, esse fato implica que, ao calcular a média desses consumos para a construção do modelo, a acurácia provavelmente será reduzida. Portanto, pode-se concluir que é necessário uma quantidade maior de dados para construir uma predição mais precisa.

# Construção do modelo

&nbsp;&nbsp;&nbsp;&nbsp;Separação dos dados base para testar o modelo, bem como seus parâmetros e especificidades.

In [ ]:
# Dividir o conjunto de dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


## Modelo - SVR 

&nbsp;&nbsp;&nbsp;&nbsp; Para mitigação de possíveis erros e melhor acurácia, foram testados outros modelos para analisar se os _outputs_ iriam diferir drasticamente um do outro. O modelo **SVR** foi um desses. O mesmo é uma variação do _**Support Vector Machine (SVM)**_, usada para regressão em vez de classificação. Ele busca encontrar uma função que se ajuste aos dados, minimizando o erro, mas de uma forma que seja robusta a outliers.

In [ ]:
modeloSvr = make_pipeline(SVR(kernel='rbf', C=10000, gamma=0.01, epsilon=0.005, tol=1e-5))

modeloSvr.fit(X_train, y_train)

&nbsp;&nbsp;&nbsp;&nbsp; Definição e ajuste de hiperparâmetros do modelo SVR usando RandomizedSearchCV.

In [ ]:
# Definindo o modelo SVR
svr = SVR()

# Definindo os hiperparâmetros a serem testados
param_distributions = {
    'C': uniform(0.01, 10000),            # Busca valores contínuos entre 0.1 e 100
    'epsilon': uniform(0.001, 1),        # Busca valores de epsilon entre 0.01 e 1
    'kernel': ['rbf'], # Busca entre os kernels mais comuns
    'gamma': uniform(0.01, 1000)          # Define como será calculado o gamma
}

# Configurando o RandomizedSearchCV
random_search = RandomizedSearchCV(svr, param_distributions, n_iter=50, cv=5, 
                                   scoring='neg_mean_squared_error', random_state=42, verbose=1)

# Treinando o RandomizedSearchCV
random_search.fit(X_train, y_train)

# Melhor conjunto de parâmetros
print("Melhores parâmetros: ", random_search.best_params_)

# Avaliando o desempenho no conjunto de teste
y_pred = random_search.best_estimator_.predict(X_test)

# Calculando as métricas
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("MSE: ", mse)
print("MAE: ", mae)
print("R2: ", r2)


Previsão dos valores de teste usando o modelo SVR e impressão dos primeiros 10 resultados.

In [ ]:
y_pred_test_svr = modeloSvr.predict(X_test)

print(y_pred_test_svr[0:10])

print(y_test[0:10] - y_pred_test_svr[0:10])


### Métricas para o modelo

&nbsp;&nbsp;&nbsp;&nbsp; A principal métrica para medir o modelo é o _**Mean Absolute Error (MAE)**_, ou Erro Médio Absoluto. Consiste no processo de medir a precisão de um modelo de previsão, calculando a média dos erros absolutos entre os valores preditos e os valores reais. 

In [ ]:
# Imprime as métricas de MAE
print(f'Métrica MAE na base de treino: {mean_absolute_error(y_train, modeloSvr.predict(X_train))}')
print(f'Métrica MAE na base de teste: {mean_absolute_error(y_test, modeloSvr.predict(X_test))}')


&nbsp;&nbsp;&nbsp;&nbsp; Como auxiliares nas métricas, também tem-se o _**Mean Squared Error (MSE)**_ e o _**Root Mean Squared Error (RMSE)**_ que também são comuns para avaliar a precisão de modelos de regressão. A principal diferença, nesse caso, é que o **MSE** e o **RMSE** penalizam erros maiores de maneira mais severa, já que os **erros são elevados ao quadrado**.

In [ ]:

# Imprime as métricas de MSE
print(f'Métrica MSE na base de treino: {mean_squared_error(y_train, modeloSvr.predict(X_train))}')
print(f'Métrica MSE na base de teste: {mean_squared_error(y_test, modeloSvr.predict(X_test))}')


In [ ]:
# Imprime as métricas de RMSE
print(f'Métrica RMSE na base de treino: {np.sqrt(mean_squared_error(y_train, modeloSvr.predict(X_train)))}')
print(f'Métrica RMSE na base de teste: {np.sqrt(mean_squared_error(y_test, modeloSvr.predict(X_test)))}')


### Gráfico para percepção visual do modelo


&nbsp;&nbsp;&nbsp;&nbsp; Para análise mais minuciosa, foi _plotado_ um gráfico de dispersão para identificar os padrões dos valores previstos em relação aos valores reais ao qual o modelo foi treinado.

In [ ]:
# Previsões para o conjunto de treino
y_train_pred = modeloSvr.predict(X_train)

# Plotando os valores reais vs previstos no conjunto de treino
plt.figure(figsize=(10, 6))

# Gráfico dos valores reais (y_train) vs valores previstos
plt.scatter(np.arange(len(y_train)), y_train, color='blue', label='Valores Reais')
plt.scatter(np.arange(len(y_train_pred)), y_train_pred, color='red', alpha=0.6, label='Valores Previstos')

plt.title('Valores Reais vs Valores Previstos (Treino)')
plt.xlabel('Índice')
plt.ylabel('Gasto')
plt.legend()
plt.show()

&nbsp;&nbsp;&nbsp;&nbsp; Gráfico de análise da regressão e como os resultados se relacionam com a **Perfeita Correspondência**.

In [ ]:
# testa o modelo com o conjunto de teste
y_pred = modeloSvr.predict(X_test)

# Previsões para o conjunto de teste
y_test_pred = modeloSvr.predict(X_test)

# Plotando os valores reais vs previstos no conjunto de teste
plt.figure(figsize=(8, 8))

# Gráfico dos valores reais vs valores previstos
plt.scatter(y_test, y_test_pred, color='blue', alpha=0.6, label='Valores Previsto x Real')

# Adicionando uma linha diagonal (y=x), que representa a "perfeita correspondência"
limite = [min(y_test.min(), y_test_pred.min()), max(y_test.max(), y_test_pred.max())]

# Adicionando a linha diagonal de correspondencia de 75% (0.75)
plt.plot(limite, 0.75*np.array(limite), color='green', linestyle='--', label='Correspondência de -75%')

# Adicionando a linha diagonal de correspondencia de 50% (0.50)
plt.plot(limite, 0.50*np.array(limite), color='orange', linestyle='--', label='Correspondência de -50%')


# Adicionando a linha diagonal de correspondencia de 75% (0.75)
plt.plot(limite, 1.5*np.array(limite), color='green', linestyle='--', label='Correspondência de +50%')

# Adicionando a linha diagonal de correspondencia de 50% (0.50)
plt.plot(limite, 1.75*np.array(limite), color='orange', linestyle='--', label='Correspondência de +75%')


plt.plot(limite, limite, color='red', linestyle='--', label='Perfeita correspondência (y=x)')
plt.title('Consumo Previsto vs Consumo Real (Teste)')
plt.xlabel('Consumo Real')
plt.ylabel('Consumo Previsto')
plt.legend()




plt.show()


# Modelo Random Forest Regressor

In [ ]:
olsmod = RandomForestRegressor(n_estimators=1000, random_state=42)
olsmod.fit(X_train, y_train)

olsres = olsmod

&nbsp;&nbsp;&nbsp;&nbsp; Verifica a distância do dado previsto com o dado real - tendo como base os dados de teste.

In [ ]:
y_pred_test = olsres.predict(X_test)

print(y_pred_test[0:10])

print(y_test[0:10] - y_pred_test[0:10])


### Métricas para o modelo

&nbsp;&nbsp;&nbsp;&nbsp; A principal métrica para medir o modelo é o _**Mean Absolute Error (MAE)**_, ou Erro Médio Absoluto. Consiste no processo de medir a precisão de um modelo de previsão, calculando a média dos erros absolutos entre os valores preditos e os valores reais. 

In [ ]:
# Imprime as métricas de MAE
print(f'Métrica MAE na base de treino: {mean_absolute_error(y_train, olsres.predict(X_train))}')
print(f'Métrica MAE na base de teste: {mean_absolute_error(y_test, olsres.predict(X_test))}')


&nbsp;&nbsp;&nbsp;&nbsp; Como auxiliares nas métricas, também tem-se o _**Mean Squared Error (MSE)**_ e o _**Root Mean Squared Error (RMSE)**_ que também são comuns para avaliar a precisão de modelos de regressão. A principal diferença, nesse caso, é que o **MSE** e o **RMSE** penalizam erros maiores de maneira mais severa, já que os **erros são elevados ao quadrado**.

In [ ]:

# Imprime as métricas de MSE
print(f'Métrica MSE na base de treino: {mean_squared_error(y_train, olsres.predict(X_train))}')
print(f'Métrica MSE na base de teste: {mean_squared_error(y_test, olsres.predict(X_test))}')


In [ ]:
# Imprime as métricas de RMSE
print(f'Métrica RMSE na base de treino: {np.sqrt(mean_squared_error(y_train, olsres.predict(X_train)))}')
print(f'Métrica RMSE na base de teste: {np.sqrt(mean_squared_error(y_test, olsres.predict(X_test)))}')


&nbsp;&nbsp;&nbsp;&nbsp; Quantidade de amostras disponíveis que o modelo ainda não viu para as treinar.

In [ ]:
# Quantidade de amostras de treino
n = X_train.shape[0]

print(f'Quantidade de amostras de treino: {n}')

### Gráfico para percepção visual do modelo

&nbsp;&nbsp;&nbsp;&nbsp; Para análise mais minuciosa, foi _plotado_ um gráfico de dispersão para identificar os padrões dos valores previstos em relação aos valores reais ao qual o modelo foi treinado.

In [ ]:
# Previsões para o conjunto de treino
y_train_pred = olsres.predict(X_train)

# Plotando os valores reais vs previstos no conjunto de treino
plt.figure(figsize=(10, 6))

# Gráfico dos valores reais (y_train) vs valores previstos
plt.scatter(np.arange(len(y_train)), y_train, color='blue', label='Valores Reais')
plt.scatter(np.arange(len(y_train_pred)), y_train_pred, color='red', alpha=0.6, label='Valores Previstos')

plt.title('Valores Reais vs Valores Previstos (Treino)')
plt.xlabel('Índice')
plt.ylabel('Gasto')
plt.legend()
plt.show()

&nbsp;&nbsp;&nbsp;&nbsp; Gráfico de análise da regressão e como os resultados se relacionam com a **Perfeita Correspondência**.

In [ ]:
# testa o modelo com o conjunto de teste
y_pred = olsres.predict(X_test)

# Previsões para o conjunto de teste
y_test_pred = olsres.predict(X_test)

# Plotando os valores reais vs previstos no conjunto de teste
plt.figure(figsize=(8, 8))

# Gráfico dos valores reais vs valores previstos
plt.scatter(y_test, y_test_pred, color='blue', alpha=0.6, label='Valores Previsto x Real')

# Adicionando uma linha diagonal (y=x), que representa a "perfeita correspondência"
limite = [min(y_test.min(), y_test_pred.min()), max(y_test.max(), y_test_pred.max())]

# Adicionando a linha diagonal de correspondencia de 75% (0.75)
plt.plot(limite, 0.75*np.array(limite), color='green', linestyle='--', label='Correspondência de -75%')

# Adicionando a linha diagonal de correspondencia de 50% (0.50)
plt.plot(limite, 0.50*np.array(limite), color='orange', linestyle='--', label='Correspondência de -50%')


# Adicionando a linha diagonal de correspondencia de 75% (0.75)
plt.plot(limite, 1.5*np.array(limite), color='green', linestyle='--', label='Correspondência de +50%')

# Adicionando a linha diagonal de correspondencia de 50% (0.50)
plt.plot(limite, 1.75*np.array(limite), color='orange', linestyle='--', label='Correspondência de +75%')


plt.plot(limite, limite, color='red', linestyle='--', label='Perfeita correspondência (y=x)')
plt.title('Consumo Previsto vs Consumo Real (Teste)')
plt.xlabel('Consumo Real')
plt.ylabel('Consumo Previsto')
plt.legend()




plt.show()


In [ ]:
# Previsões para o conjunto de treino
y_test_pred = olsres.predict(X_test)

# Plotando os valores reais vs previstos no conjunto de treino
plt.figure(figsize=(10, 6))

# Gráfico dos valores reais (y_test) vs valores previstos
plt.scatter(np.arange(len(y_test)), y_test, color='blue', label='Valores Reais')
plt.scatter(np.arange(len(y_test_pred)), y_test_pred, color='red', alpha=0.6, label='Valores Previstos')

plt.title('Valores Reais vs Valores Previstos (Teste)')
plt.xlabel('Índice')
plt.ylabel('Gasto')
plt.legend()
plt.show()

## Comparacao de modelos SVR x RandomForestRegressor

&nbsp;&nbsp;&nbsp;&nbsp; Gráfico de comparação entre os modelos SVR e RandomForestRegressor.

In [ ]:
# grafico de comparacao entre os modelos de regressao svr e random forest
plt.figure(figsize=(12, 6))
plt.scatter(y_test, y_pred, color='blue', alpha=0.6, label='Random Forest')
plt.scatter(y_test, modeloSvr.predict(X_test), color='red', alpha=0.6, label='SVR')
plt.plot(y_test, y_test, color='blue', linestyle='--', label='Perfeita correspondência (y=x)')
# Adicionando a linha diagonal de correspondencia de 75% (0.75)
plt.plot(limite, 0.75*np.array(limite), color='green', linestyle='--', label='Correspondência de -75%')

# Adicionando a linha diagonal de correspondencia de 50% (0.50)
plt.plot(limite, 0.50*np.array(limite), color='orange', linestyle='--', label='Correspondência de -50%')


# Adicionando a linha diagonal de correspondencia de 75% (0.75)
plt.plot(limite, 1.5*np.array(limite), color='green', linestyle='--', label='Correspondência de +50%')

# Adicionando a linha diagonal de correspondencia de 50% (0.50)
plt.plot(limite, 1.75*np.array(limite), color='orange', linestyle='--', label='Correspondência de +75%')

plt.title('Comparação entre Random Forest e SVR - Client Code 683')
plt.xlabel('Gasto Real')
plt.ylabel('Gasto Previsto')
plt.legend()
plt.show()


&nbsp;&nbsp;&nbsp;&nbsp; Métricas de avaliação dos modelos, como MAE, MSE e RMSE.

In [ ]:
# comparacao entre os modelos de regressao svr e random forest

# R2, MAE, MSE, RMSE

# Random Forest
r2_rf = r2_score(y_test, y_pred)
mae_rf = mean_absolute_error(y_test, y_pred)
mse_rf = mean_squared_error(y_test, y_pred)
rmse_rf = np.sqrt(mse_rf)

# SVR
r2_svr = r2_score(y_test, modeloSvr.predict(X_test))
mae_svr = mean_absolute_error(y_test, modeloSvr.predict(X_test))
mse_svr = mean_squared_error(y_test, modeloSvr.predict(X_test))
rmse_svr = np.sqrt(mse_svr)

# Criando um DataFrame com as métricas
df_metricas = pd.DataFrame({
    'Random Forest': [r2_rf, mae_rf, mse_rf, rmse_rf],
    'SVR': [r2_svr, mae_svr, mse_svr, rmse_svr]
}, index=['R2', 'MAE', 'MSE', 'RMSE'])

df_metricas

## Preparação do dataset para o Kmeans

&nbsp;&nbsp;&nbsp;&nbsp; Ao realizarmos as predições de consumo por cada cliente, a equipe decidiu clusterizá-los com base em suas especificidades e perfil de consumo. Para isso, fez-se necessário gerar um novo modelo não-supervisionado (_**Kmeans**_) para encontrar padrões e proximidade entre os clientes.

&nbsp;&nbsp;&nbsp;&nbsp; Leitura abaixo do _**csv**_ atualizado com os dados metereológicos.

In [ ]:
df_1 = pd.read_csv("../data/df_atualizado.csv")

&nbsp;&nbsp;&nbsp;&nbsp; Leitura abaixo do _**csv**_ com os dados IPCA de Porto Alegre.

In [ ]:
df_porto_alegre = pd.read_csv('../data/porto_alegre_dados_finalizados.csv')

&nbsp;&nbsp;&nbsp;&nbsp; Substituição da coluna 'Mês' por 'mes' para que mantenha o padrão anterior.

In [ ]:
df_porto_alegre = df_porto_alegre.rename(columns={'Mês': 'mes'})

&nbsp;&nbsp;&nbsp;&nbsp; Merge de ambos os _**csv's**_ para agregar mais festures ao modelo.

In [ ]:
df = pd.merge(df_1, df_porto_alegre, on='mes', how='left')
df

&nbsp;&nbsp;&nbsp;&nbsp; Drop da coluna 'date' pois já existem colunas de mês, hora e dia da semana.

In [ ]:
# df = df.drop(columns = ['date'])

&nbsp;&nbsp;&nbsp;&nbsp; Renomeação do nome de algumas colunas para maneter o padrão anterior.

In [ ]:
df = df.rename(columns={'1.Alimentação e bebidas': 'alimentacao/bebidas', '2.AHabitação': 'habitacao', '3.Artigos de residência': 'artigo-de-residencia', '4.Vestuários': 'vestuario', '5.Transportes': 'transportes' , '6.Saúde e cuidados pessoais': 'saude/cuidados', '7.Despesas pessoais': 'despesas-pessoais', '8.Educação': 'educacao', '9.Comunicação': 'comunicacao', 'Índice geral': 'indice-geral'})

&nbsp;&nbsp;&nbsp;&nbsp; Criação de uma função para remover _outliers_ com base e desvios padrões.

In [ ]:
def remove_outliers(data, std_devs=3):
    mean = np.mean(data, axis=0)
    std_dev = np.std(data, axis=0)
    lower_bound = mean - std_devs * std_dev
    upper_bound = mean + std_devs * std_dev
    mask = np.all((data >= lower_bound) & (data <= upper_bound), axis=1)
    return data[mask], mask

&nbsp;&nbsp;&nbsp;&nbsp; Exclusão dos valores negativos encontrados na coluna de gastos pois, possivelmente, se tratam de anomalias e não são relevantes para a predição do consumo.

In [ ]:
negativos = df[df['gasto'] < 0]
print(f"Valores negativos encontrados:\n{negativos}")

df = df[df['gasto'] >= 0]

&nbsp;&nbsp;&nbsp;&nbsp; Plotagem do gráfico do **Método do Cotovelo** para identificar a melhor quantidade de clusters que o modelo deve apresentar e se estruturas.

In [ ]:
X, _ = make_blobs(n_samples=300, centers=4, cluster_std=0.60, random_state=0)

sse = []

K = range(1, 11)
for k in K:
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(X)
    sse.append(kmeans.inertia_)

plt.figure(figsize=(10, 6))
plt.plot(K, sse, marker='o')
plt.title('Método do Cotovelo')
plt.xlabel('Número de Clusters')
plt.ylabel('Soma das Distâncias Quadráticas Dentro dos Clusters (SSE)')
plt.xticks(K)
plt.grid()
plt.show()

&nbsp;&nbsp;&nbsp;&nbsp; Pela curva não ser tão brusca, houve a dúvida entre 4 ou 5 _clusters_ que, posteriormente, seriam testados no código do _**Kmeans**_ e plotados em um gráfico _**PCA**_.

In [ ]:
df.head(20)

## Kmeans

&nbsp;&nbsp;&nbsp;&nbsp; Geração do Modelo _**Kmeans**_ com _clusters_ indo em uma faixa de 2-6 e plotagem de gráficos 2D e 3D para melhor visualização da separação de cada grupo encontrado e como o mesmo se comporta no espaço.

In [ ]:
cluster_range = range(2, 7)
rotation_angles = range(0, 360, 90)

scaler = StandardScaler()
df_scaled = scaler.fit_transform(df.drop(columns=['date', 'clientCode', 'clientIndex']))

df_scaled_no_outliers, mask = remove_outliers(df_scaled, std_devs=3)

for clusters in cluster_range:
    
    km = KMeans(n_clusters=clusters, random_state=42)
    y_predict_no_outliers = km.fit_predict(df_scaled_no_outliers)

    pca = PCA(n_components=3)
    df_pca_3d = pca.fit_transform(df_scaled_no_outliers)

    pca_2d = PCA(n_components=2)
    df_pca_2d = pca_2d.fit_transform(df_scaled_no_outliers)

    fig = plt.figure(figsize=(14, 7))
    ax2 = fig.add_subplot(122)

    sns.scatterplot(x=df_pca_2d[:, 0], y=df_pca_2d[:, 1], hue=y_predict_no_outliers, s=100, alpha=0.75, 
                    palette="Set1", edgecolor='white', linewidth=0.8, ax=ax2)

    ax2.set_title(f"2D Clusters: {clusters}")
    ax2.set_xlabel("Componente Principal 1")
    ax2.set_ylabel("Componente Principal 2")
    ax2.grid(True)

    for angle in rotation_angles:
        
        fig_3d = plt.figure(figsize=(14, 7))
        ax = fig_3d.add_subplot(111, projection='3d')

        scatter_3d = ax.scatter(df_pca_3d[:, 0], df_pca_3d[:, 1], df_pca_3d[:, 2], 
                                c=y_predict_no_outliers, s=100, alpha=0.75, cmap='Set1', linewidth=0.8)

        ax.set_title(f"3D Clusters: {clusters}, Ângulo: {angle}°")
        ax.set_xlabel("Componente Principal 1")
        ax.set_ylabel("Componente Principal 2")
        ax.set_zlabel("Componente Principal 3")

        ax.view_init(elev=30, azim=angle)

        plt.show()

df['cluster'] = 0  

for clusters in cluster_range:
    km = KMeans(n_clusters=clusters, random_state=42)
    y_predict_no_outliers = km.fit_predict(df_scaled_no_outliers)

    
    if clusters == 5:
        df.loc[mask, 'cluster'] = y_predict_no_outliers  

&nbsp;&nbsp;&nbsp;&nbsp; Com base nos resultados, o número de _clusters_ escolhido foi: **5**.

&nbsp;&nbsp;&nbsp;&nbsp; Abaixo, estatísticas de gasto por cluster para identificar comportamento de consumo.

In [ ]:
perfil_consumo = df.groupby('cluster')['gasto'].agg(['mean', 'median', 'std', 'min', 'max', 'count']).reset_index()

print(perfil_consumo)

**Cluster 0**
O Cluster 0 tem uma média de consumo de 3,63 m³ e uma mediana de 0,40 m³, o que sugere que a maioria dos clientes consome muito pouco. No entanto, o desvio padrão extremamente alto de 192,99 m³ revela uma grande variação, causada por outliers com valores de consumo máximos de até 39.382,09 m³. Isso indica que, embora a maioria consuma pouco, alguns clientes têm um consumo excepcionalmente elevado, aumentando a variabilidade do grupo.

**Cluster 1**
O Cluster 1 apresenta uma média de consumo de 0,83 m³ e uma mediana de 0,41 m³, indicando também um padrão de consumo baixo. O desvio padrão de 3,30 m³ sugere que há uma variabilidade menor do que no Cluster 0, mas ainda existem alguns consumidores com consumo bem maior, com um máximo de 277,71 m³. A maioria dos clientes está concentrada em pequenos consumos, mas ainda há outliers que elevam a variabilidade.

**Cluster 2**
O Cluster 2 possui uma média de consumo de 0,73 m³ e uma mediana de 0,25 m³, destacando um perfil de consumo ainda mais baixo. O desvio padrão de 3,19 m³ reflete uma variabilidade razoável, com o valor máximo chegando a 211,54 m³. Este cluster é formado majoritariamente por consumidores de baixo consumo, mas há alguns com consumos maiores.

**Cluster 3**
O Cluster 3 tem uma média de consumo de 0,80 m³ e uma mediana de 0,24 m³, com uma variabilidade (desvio padrão de 3,15 m³) um pouco menor, mas ainda relevante. Alguns clientes têm consumo de até 158,08 m³, demonstrando a presença de outliers, embora a maioria mantenha um padrão de consumo reduzido.

**Cluster 4**
O Cluster 4 tem uma média de 0,74 m³ e uma mediana de 0,30 m³, e é o cluster com a menor variabilidade (desvio padrão de 2,93 m³). O consumo máximo é de 156,52 m³, indicando menos casos de consumo extremo comparado aos outros clusters. Aqui, o consumo é mais concentrado em valores baixos, com menos outliers.


&nbsp;&nbsp;&nbsp;&nbsp; Por conta do _dataset_ ser por hora, há clientes com mais de um linha de dado e que, por conta da variação de consumo, assumia mais de um _cluster_. Para corrgir esse erro, agrupamos os _clusters_ por _clientCode_ e mudamos o valor para a moda de _cluster_ que mais apareceu naquele _clientCodeCode_ específico. Assim, cada cliente correspondia à apenas um grupo.

In [ ]:

cluster_mais_frequente = df.groupby('clientCode')['cluster'].agg(lambda x: x.value_counts().idxmax())

df['cluster'] = df['clientCode'].map(cluster_mais_frequente)

print(df)

&nbsp;&nbsp;&nbsp;&nbsp; Contagem da quantidade de linhas com cada _cluster_ específico.

In [ ]:
count_ones = df['cluster'].value_counts()[3]
print(f"A quantidade de números x é: {count_ones}")

&nbsp;&nbsp;&nbsp;&nbsp; Criação de um novo _dataframe_ com todas novas features e cada cliente rotulado por seu respectivo _cluster_.

In [ ]:
df.to_csv('../data/df_cluster.csv', index=False)

# Modelo preditivo para previsão de consumo com base nos clusters

## Importação dos dados

In [ ]:
df = pd.read_csv('../data/df_cluster.csv')
df.columns

In [ ]:
df

## Tratamento prévio

&nbsp;&nbsp;&nbsp;&nbsp; Tendo em vista que cada tipo de modelo exige um tratamento de dados específico, para o presente modelo foram removidas as colunas 'medidor' e 'gasto_por_hora'. Nesse sentido, optou-se por utilizar apenas a coluna 'gasto' no treinamento, já que ambas se referem ao consumo. Assim, o valor de 'gasto' é obtido pela multiplicação do pulseCount pelo gain de cada medição, enquanto 'gasto_por_hora' representa o consumo por hora.

In [ ]:
df = df.drop(columns=['medidor','gasto_por_hora', 'hora'])

&nbsp;&nbsp;&nbsp;&nbsp; Além disso, foram removidos os valores negativos presentes no *DataFrame*, garantindo que tais valores não interfiram no processo de treinamento do modelo. Uma vez que a presença de dados negativos poderia causar distorções nos resultados, especialmente em variáveis que deveriam representar quantidades positivas, como consumo, que naturalmente não assumem valores abaixo de zero.

In [ ]:
df = df[df['gasto'] >= 0]
len(df[df['gasto'] < 0])

&nbsp;&nbsp;&nbsp;&nbsp; Com o objetivo de aprimorar o desempenho dos modelos, decidiu-se dividir o *DataFrame* original em cinco subconjuntos, cada um correspondente a um cluster gerado pelo algoritmo K-means, que agrupou os dados de acordo com o perfil de consumo. Dessa forma, cada subconjunto será utilizado para treinar um modelo específico, o que permitirá uma abordagem mais personalizada para cada grupo de perfis de consumo. Assim, podendo gerar melhores desempenhos nas previsões ao capturar as características específicas de cada cluster.

In [ ]:
df_0 = df[df['cluster'] == 0]
df_1 = df[df['cluster'] == 1]
df_2 = df[df['cluster'] == 2]
df_3 = df[df['cluster'] == 3]
df_4 = df[df['cluster'] == 4]


In [ ]:
df_2.to_csv('../data/df_2.csv', index=False)

&nbsp;&nbsp;&nbsp;&nbsp; Por fim, os outliers foram removidos ao identificar valores de consumo de gás que se desviavam excessivamente da média, utilizando o desvio padrão para definir os limites superior e inferior. Dessa forma, os dados que ultrapassavam esses limites (valores muito acima ou abaixo da média) foram eliminados, mantendo apenas os consumos dentro de um desvio padrão da média.

In [ ]:
# Identifica o consumo medio do cliente e o desvio padrao
media = df_0['gasto'].mean()
desvio = df_0['gasto'].std()

# Remove os consumos que estao fora do desvio padrao

maximo = media + (1*desvio)
df_0 = df_0[df_0['gasto'] < maximo]

minimo = media - (1*desvio)
df_0 = df_0[df_0['gasto'] > minimo]

In [ ]:
# Identifica o consumo medio do cliente e o desvio padrao
media = df_1['gasto'].mean()
desvio = df_1['gasto'].std()

# Remove os consumos que estao fora do desvio padrao

maximo = media + (1*desvio)
df_1 = df_1[df_1['gasto'] < maximo]

minimo = media - (1*desvio)
df_1 = df_1[df_1['gasto'] > minimo]


In [ ]:
# Identifica o consumo medio do cliente e o desvio padrao
media = df_2['gasto'].mean()
desvio = df_2['gasto'].std()

# Remove os consumos que estao fora do desvio padrao

maximo = media + (1*desvio)
df_2 = df_2[df_2['gasto'] < maximo]

minimo = media - (1*desvio)
df_2 = df_2[df_2['gasto'] > minimo]


In [ ]:
# Identifica o consumo medio do cliente e o desvio padrao
media = df_3['gasto'].mean()
desvio = df_3['gasto'].std()

# Remove os consumos que estao fora do desvio padrao

maximo = media + (1*desvio)
df_3 = df_3[df_3['gasto'] < maximo]

minimo = media - (1*desvio)
df_3 = df_3[df_3['gasto'] > minimo]


In [ ]:
# Identifica o consumo medio do cliente e o desvio padrao
media = df_4['gasto'].mean()
desvio = df_4['gasto'].std()

# Remove os consumos que estao fora do desvio padrao

maximo = media + (1*desvio)
df_4 = df_4[df_4['gasto'] < maximo]

minimo = media - (1*desvio)
df_4 = df_4[df_4['gasto'] > minimo]


## Modelo do cluster 0 - XGBoost Regressor

### Treinamento do modelo sem os hiperparâmetros

In [ ]:
X = df_0.drop(columns=['gasto','date'])  
y = df_0['gasto'] 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

model_0 = XGBRegressor(objective='reg:pseudohubererror', random_state=123)

model_0.fit(X_train, y_train)

predictions = model_0.predict(X_test)

#### Métricas e gráficos da perfomace do modelo

In [ ]:
# Plotando os valores reais vs previstos no conjunto de teste
plt.figure(figsize=(8, 8))

# Gráfico dos valores reais vs valores previstos
plt.scatter(y_test, predictions, color='blue', alpha=0.6, label='Valores Previsto x Real')

# Definindo os limites para as linhas de correspondência
limite = [min(y_test.min(), predictions.min()), max(y_test.max(), predictions.max())]

# Adicionando a linha diagonal de correspondencia de 75% (0.75)
plt.plot(limite, 0.75*np.array(limite), color='green', linestyle='--', label='Correspondência de -75%')

# Adicionando a linha diagonal de correspondencia de 50% (0.50)
plt.plot(limite, 0.50*np.array(limite), color='orange', linestyle='--', label='Correspondência de -50%')

# Adicionando a linha diagonal de correspondencia de 75% (0.75)
plt.plot(limite, 1.5*np.array(limite), color='green', linestyle='--', label='Correspondência de +50%')

# Adicionando a linha diagonal de correspondencia de 50% (0.50)
plt.plot(limite, 1.75*np.array(limite), color='orange', linestyle='--', label='Correspondência de +75%')

# Linha de perfeita correspondência (y = x)
plt.plot(limite, limite, color='red', linestyle='--', label='Perfeita correspondência (y=x)')

plt.title('Valores Reais vs Valores Previstos no cluster 0')
plt.xlabel('Consumo Real')
plt.ylabel('Consumo Previsto')
plt.legend()
plt.show()

In [ ]:
# Imprime as métricas de MAE
print(f'Métrica MAE na base de teste: {mean_absolute_error(y_test, predictions):.4f}')

# Imprime as métricas de MSE
print(f'Métrica MSE na base de teste: {mean_squared_error(y_test, predictions):.4f}')

# Imprime as métricas de RMSE
print(f'Métrica RMSE na base de teste: {np.sqrt(mean_squared_error(y_test, predictions)):.4f}')

# Imprime o R²
print(f'Métrica R² na base de teste: {r2_score(y_test, predictions):.4f}')

### Salvando o modelo

In [ ]:
# Save the model to a file
joblib.dump(model_0, 'xgb_regressor_model0.pkl')
# Load the model later (when needed)
loaded_model_0 = joblib.load('xgb_regressor_model0.pkl')
# Predict using the loaded model
predictions = loaded_model_0.predict(X_test)

## Modelo do cluster 1 - XGBoost Regressor

### Treinamento do modelo sem os hiperparâmetros

In [ ]:
X = df_1.drop(columns=['gasto','date'])  
y = df_1['gasto'] 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

model_1 = XGBRegressor(objective='reg:pseudohubererror', random_state=123)

model_1.fit(X_train, y_train)

predictions = model_1.predict(X_test)

#### Métricas e gráficos da perfomace do modelo

In [ ]:
# Plotando os valores reais vs previstos no conjunto de teste
plt.figure(figsize=(8, 8))

# Gráfico dos valores reais vs valores previstos
plt.scatter(y_test, predictions, color='blue', alpha=0.6, label='Valores Previsto x Real')

# Definindo os limites para as linhas de correspondência
limite = [min(y_test.min(), predictions.min()), max(y_test.max(), predictions.max())]

# Adicionando a linha diagonal de correspondencia de 75% (0.75)
plt.plot(limite, 0.75*np.array(limite), color='green', linestyle='--', label='Correspondência de -75%')

# Adicionando a linha diagonal de correspondencia de 50% (0.50)
plt.plot(limite, 0.50*np.array(limite), color='orange', linestyle='--', label='Correspondência de -50%')

# Adicionando a linha diagonal de correspondencia de 75% (0.75)
plt.plot(limite, 1.5*np.array(limite), color='green', linestyle='--', label='Correspondência de +50%')

# Adicionando a linha diagonal de correspondencia de 50% (0.50)
plt.plot(limite, 1.75*np.array(limite), color='orange', linestyle='--', label='Correspondência de +75%')

# Linha de perfeita correspondência (y = x)
plt.plot(limite, limite, color='red', linestyle='--', label='Perfeita correspondência (y=x)')

plt.title('Valores Reais vs Valores Previstos no cluster 1')
plt.xlabel('Consumo Real')
plt.ylabel('Consumo Previsto')
plt.legend()
plt.show()

In [ ]:
# Imprime as métricas de MAE
print(f'Métrica MAE na base de teste: {mean_absolute_error(y_test, predictions):.4f}')

# Imprime as métricas de MSE
print(f'Métrica MSE na base de teste: {mean_squared_error(y_test, predictions):.4f}')

# Imprime as métricas de RMSE
print(f'Métrica RMSE na base de teste: {np.sqrt(mean_squared_error(y_test, predictions)):.4f}')

# Imprime o R²
print(f'Métrica R² na base de teste: {r2_score(y_test, predictions):.4f}')

### Salvando o modelo

In [ ]:
# Save the model to a file
joblib.dump(model_1, 'xgb_regressor_model1.pkl')
# Load the model later (when needed)
loaded_model_1 = joblib.load('xgb_regressor_model1.pkl')
# Predict using the loaded model
predictions = loaded_model_1.predict(X_test)

## Modelo do cluster 2 - XGBoost Regressor

### Treinamento do modelo sem os hiperparâmetros

In [ ]:
X = df_2.drop(columns=['gasto','date'])  
y = df_2['gasto'] 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

model_2 = XGBRegressor(objective='reg:pseudohubererror', random_state=123)

model_2.fit(X_train, y_train)

predictions = model_2.predict(X_test)

#### Métricas e gráficos da perfomace do modelo

In [ ]:
# Plotando os valores reais vs previstos no conjunto de teste
plt.figure(figsize=(8, 8))

# Gráfico dos valores reais vs valores previstos
plt.scatter(y_test, predictions, color='blue', alpha=0.6, label='Valores Previsto x Real')

# Definindo os limites para as linhas de correspondência
limite = [min(y_test.min(), predictions.min()), max(y_test.max(), predictions.max())]

# Adicionando a linha diagonal de correspondencia de 75% (0.75)
plt.plot(limite, 0.75*np.array(limite), color='green', linestyle='--', label='Correspondência de -75%')

# Adicionando a linha diagonal de correspondencia de 50% (0.50)
plt.plot(limite, 0.50*np.array(limite), color='orange', linestyle='--', label='Correspondência de -50%')

# Adicionando a linha diagonal de correspondencia de 75% (0.75)
plt.plot(limite, 1.5*np.array(limite), color='green', linestyle='--', label='Correspondência de +50%')

# Adicionando a linha diagonal de correspondencia de 50% (0.50)
plt.plot(limite, 1.75*np.array(limite), color='orange', linestyle='--', label='Correspondência de +75%')

# Linha de perfeita correspondência (y = x)
plt.plot(limite, limite, color='red', linestyle='--', label='Perfeita correspondência (y=x)')

plt.title('Valores Reais vs Valores Previstos no cluster 2')
plt.xlabel('Consumo Real')
plt.ylabel('Consumo Previsto')
plt.legend()
plt.show()

In [ ]:
# Imprime as métricas de MAE
print(f'Métrica MAE na base de teste: {mean_absolute_error(y_test, predictions):.4f}')

# Imprime as métricas de MSE
print(f'Métrica MSE na base de teste: {mean_squared_error(y_test, predictions):.4f}')

# Imprime as métricas de RMSE
print(f'Métrica RMSE na base de teste: {np.sqrt(mean_squared_error(y_test, predictions)):.4f}')

# Imprime o R²
print(f'Métrica R² na base de teste: {r2_score(y_test, predictions):.4f}')

### Salvando o modelo

In [ ]:
# Save the model to a file
joblib.dump(model_2, 'xgb_regressor_model2.pkl')
# Load the model later (when needed)
loaded_model_2 = joblib.load('xgb_regressor_model2.pkl')
# Predict using the loaded model
predictions = loaded_model_2.predict(X_test)

## Modelo do cluster 3 - XGBoost Regressor

### Treinamento do modelo sem os hiperparâmetros

&nbsp;&nbsp;&nbsp;&nbsp; Para uma primeira análise, utilizou o XGBoost Regressor que é um algoritmo de aprendizado supervisionado baseado em árvores de decisão. Nesse sentido, ele faz parte de uma técnica chamada boosting, na qual várias árvores de decisão são criadas de maneira sequencial. Assim, cada nova árvore tenta corrigir os erros das anteriores, melhorando gradualmente a precisão do modelo. Além disso, nesta etapa inicial, os hiperparâmetros não foram ajustados.

In [ ]:
X = df_3.drop(columns=['gasto','date'])  
y = df_3['gasto'] 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

model_3 = XGBRegressor(objective='reg:pseudohubererror', random_state=123)

model_3.fit(X_train, y_train)

predictions = model_3.predict(X_test)

#### Métricas e gráficos da perfomace do modelo

&nbsp;&nbsp;&nbsp;&nbsp; Com a finalidade de obsersar a análise e a performace do modelo, plotou-se uma gráfico de dispersão que compara os valores reais (y_test) com os valores previstos pelo modelo (predictions). Além disso, ele adiciona diversas linhas de referência para visualização de correspondências entre os valores. Com isso, esse tipo de gráfico é uma ferramenta visual para identificar o quão bem o modelo se ajusta aos dados observados.

In [ ]:
# Plotando os valores reais vs previstos no conjunto de teste
plt.figure(figsize=(8, 8))

# Gráfico dos valores reais vs valores previstos
plt.scatter(y_test, predictions, color='blue', alpha=0.6, label='Valores Previsto x Real')

# Definindo os limites para as linhas de correspondência
limite = [min(y_test.min(), predictions.min()), max(y_test.max(), predictions.max())]

# Adicionando a linha diagonal de correspondencia de 75% (0.75)
plt.plot(limite, 0.75*np.array(limite), color='green', linestyle='--', label='Correspondência de -75%')

# Adicionando a linha diagonal de correspondencia de 50% (0.50)
plt.plot(limite, 0.50*np.array(limite), color='orange', linestyle='--', label='Correspondência de -50%')

# Adicionando a linha diagonal de correspondencia de 75% (0.75)
plt.plot(limite, 1.5*np.array(limite), color='green', linestyle='--', label='Correspondência de +50%')

# Adicionando a linha diagonal de correspondencia de 50% (0.50)
plt.plot(limite, 1.75*np.array(limite), color='orange', linestyle='--', label='Correspondência de +75%')

# Linha de perfeita correspondência (y = x)
plt.plot(limite, limite, color='red', linestyle='--', label='Perfeita correspondência (y=x)')

plt.title('Valores Reais vs Valores Previstos no cluster 3')
plt.xlabel('Consumo Real')
plt.ylabel('Consumo Previsto')
plt.legend()
plt.show()

&nbsp;&nbsp;&nbsp;&nbsp; O gráfico mencionado foi gerado com base no modelo de cluster 3, e sua análise permite observar que a maioria dos dados se concentra nas regiões delimitadas pelas linhas de referência de 75% e 50%. Dessa forma, esse fato indica que o modelo está fazendo previsões consistentes, com uma certa aproximação em relação aos valores reais.

&nbsp;&nbsp;&nbsp;&nbsp; Foram utilizadas as métricas MAE, MSE, RMSE e R² para avaliar a performance de modelo. Segue abaixo os resultados obtidos nesta primeira análise.

In [ ]:
# Imprime as métricas de MAE
print(f'Métrica MAE na base de teste: {mean_absolute_error(y_test, predictions):.4f}')

# Imprime as métricas de MSE
print(f'Métrica MSE na base de teste: {mean_squared_error(y_test, predictions):.4f}')

# Imprime as métricas de RMSE
print(f'Métrica RMSE na base de teste: {np.sqrt(mean_squared_error(y_test, predictions)):.4f}')

# Imprime o R²
print(f'Métrica R² na base de teste: {r2_score(y_test, predictions):.4f}')


### Ajuste hiperparâmetros

&nbsp;&nbsp;&nbsp;&nbsp; Com o objetivo de melhorar a performance do modelo, foi realizado um processo de otimização de hiperparâmetros utilizando a biblioteca Optuna. Essa biblioteca permite realizar uma busca eficiente pelos melhores hiperparâmetros com o objetivo de identificar as combinações que oferecem os melhores resultados

In [ ]:
def objective(trial):
    params = {
        "objective": "reg:pseudohubererror",  # Mantendo o mesmo objetivo do primeiro código
        "n_estimators": 1000,
        "verbosity": 0,
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
        "max_depth": trial.suggest_int("max_depth", 10, 150),
        "subsample": trial.suggest_float("subsample", 0.05, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.05, 1.0),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 20),
        "random_state": 123  # Garantindo reprodutibilidade
    }

    model_3 = XGBRegressor(**params)
    model_3.fit(X_train, y_train)

    predictions = model_3.predict(X_test)
    r2 = r2_score(y_test, predictions)
    
    return r2

In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=15)

In [ ]:
print('Melhores hiperparâmetros:', study.best_params)

### Treinamento do modelo com os hiperparâmetros

&nbsp;&nbsp;&nbsp;&nbsp; Após identificar os melhores hiperparâmetros através do processo de otimização com a biblioteca Optuna, esses parâmetros foram aplicados ao treinamento do modelo. O objetivo dessa nova etapa de treinamento é verificar se as otimizações resultaram em uma melhora perceptível na performance do modelo

In [ ]:
model_3 = XGBRegressor(objective='reg:pseudohubererror', random_state=123, colsample_bytree =0.6304272148495176, learning_rate =0.002729517529305024, max_depth= 80, n_estimators= 1000, subsample=0.8659933074934297, min_child_weight= 1)
model_3.fit(X_train, y_train)
predictions = model_3.predict(X_test)

#### Métricas e gráficos da performace do modelo

&nbsp;&nbsp;&nbsp;&nbsp; Após o treinamento do modelo utilizando os hiperparâmetros otimizados, foi gerado um gráfico para visualizar a performance das previsões. Esse gráfico é necessário para compreender como as otimizações influenciaram o desempenho do modelo, fornecendo uma comparação visual entre os valores reais e os valores previstos.



In [ ]:
# Plotando os valores reais vs previstos no conjunto de teste
plt.figure(figsize=(8, 8))

# Gráfico dos valores reais vs valores previstos
plt.scatter(y_test, predictions, color='blue', alpha=0.6, label='Valores Previsto x Real')

# Definindo os limites para as linhas de correspondência
limite = [min(y_test.min(), predictions.min()), max(y_test.max(), predictions.max())]

# Adicionando a linha diagonal de correspondencia de 75% (0.75)
plt.plot(limite, 0.75*np.array(limite), color='green', linestyle='--', label='Correspondência de -75%')

# Adicionando a linha diagonal de correspondencia de 50% (0.50)
plt.plot(limite, 0.50*np.array(limite), color='orange', linestyle='--', label='Correspondência de -50%')

# Adicionando a linha diagonal de correspondencia de 75% (0.75)
plt.plot(limite, 1.5*np.array(limite), color='green', linestyle='--', label='Correspondência de +50%')

# Adicionando a linha diagonal de correspondencia de 50% (0.50)
plt.plot(limite, 1.75*np.array(limite), color='orange', linestyle='--', label='Correspondência de +75%')

# Linha de perfeita correspondência (y = x)
plt.plot(limite, limite, color='red', linestyle='--', label='Perfeita correspondência (y=x)')

plt.title('Valores Reais vs Valores Previstos no cluster 3')
plt.xlabel('Consumo Real')
plt.ylabel('Consumo Previsto')
plt.legend()
plt.show()

&nbsp;&nbsp;&nbsp;&nbsp; Assim, observa-se as seguintes métricas obtidas após os ajustes dos hiperparâmetros.

In [ ]:
# Imprime as métricas de MAE
print(f'Métrica MAE na base de teste: {mean_absolute_error(y_test, predictions):.4f}')

# Imprime as métricas de MSE
print(f'Métrica MSE na base de teste: {mean_squared_error(y_test, predictions):.4f}')

# Imprime as métricas de RMSE
print(f'Métrica RMSE na base de teste: {np.sqrt(mean_squared_error(y_test, predictions)):.4f}')

# Imprime o R²
print(f'Métrica R² na base de teste: {r2_score(y_test, predictions):.4f}')

### Salvando o modelo

In [ ]:
# Save the model to a file
joblib.dump(model_3, 'xgb_regressor_model3.pkl')
# Load the model later (when needed)
loaded_model_3 = joblib.load('xgb_regressor_model3.pkl')
# Predict using the loaded model
predictions = loaded_model_3.predict(X_test)

## Modelo do cluster 4 - XGBoost Regressor

### Treinamento do modelo sem os hiperparâmetros

In [ ]:
X = df_4.drop(columns=['gasto','date'])  
y = df_4['gasto'] 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

model_4 = XGBRegressor(objective='reg:pseudohubererror', random_state=123)

model_4.fit(X_train, y_train)

predictions = model_4.predict(X_test)

#### Métricas e gráficos da perfomace do modelo

In [ ]:
# Plotando os valores reais vs previstos no conjunto de teste
plt.figure(figsize=(8, 8))

# Gráfico dos valores reais vs valores previstos
plt.scatter(y_test, predictions, color='blue', alpha=0.6, label='Valores Previsto x Real')

# Definindo os limites para as linhas de correspondência
limite = [min(y_test.min(), predictions.min()), max(y_test.max(), predictions.max())]

# Adicionando a linha diagonal de correspondencia de 75% (0.75)
plt.plot(limite, 0.75*np.array(limite), color='green', linestyle='--', label='Correspondência de -75%')

# Adicionando a linha diagonal de correspondencia de 50% (0.50)
plt.plot(limite, 0.50*np.array(limite), color='orange', linestyle='--', label='Correspondência de -50%')

# Adicionando a linha diagonal de correspondencia de 75% (0.75)
plt.plot(limite, 1.5*np.array(limite), color='green', linestyle='--', label='Correspondência de +50%')

# Adicionando a linha diagonal de correspondencia de 50% (0.50)
plt.plot(limite, 1.75*np.array(limite), color='orange', linestyle='--', label='Correspondência de +75%')

# Linha de perfeita correspondência (y = x)
plt.plot(limite, limite, color='red', linestyle='--', label='Perfeita correspondência (y=x)')

plt.title('Valores Reais vs Valores Previstos no cluster 4')
plt.xlabel('Consumo Real')
plt.ylabel('Consumo Previsto')
plt.legend()
plt.show()

In [ ]:
# Imprime as métricas de MAE
print(f'Métrica MAE na base de teste: {mean_absolute_error(y_test, predictions):.4f}')

# Imprime as métricas de MSE
print(f'Métrica MSE na base de teste: {mean_squared_error(y_test, predictions):.4f}')

# Imprime as métricas de RMSE
print(f'Métrica RMSE na base de teste: {np.sqrt(mean_squared_error(y_test, predictions)):.4f}')

# Imprime o R²
print(f'Métrica R² na base de teste: {r2_score(y_test, predictions):.4f}')

### Salvando o modelo

In [ ]:
# Save the model to a file
joblib.dump(model_4, 'xgb_regressor_model4.pkl')
# Load the model later (when needed)
loaded_model_4 = joblib.load('xgb_regressor_model4.pkl')
# Predict using the loaded model
predictions = loaded_model_4.predict(X_test)